In [5]:
import Pkg;
Pkg.add("CSV"); 
Pkg.add("DataFrames");

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [92]:
using CSV
using DataFrames
using LinearAlgebra

In [101]:
function get_actual_values_from_cumulatives(cumulative_vector)
    shifted_values = zeros(Float64, size(cumulative_vector)[1]+1)
    for (index, value) in enumerate(cumulative_vector)
       shifted_values[index+1] = value  
    end

    final_values = zeros(Float64, size(cumulative_vector)[1])
    for (index, value) in enumerate(cumulative_vector)
       final_values[index] = -shifted_values[index]+cumulative_vector[index]  
    end
    return final_values
end

get_actual_values_from_cumulatives (generic function with 1 method)

Read data. Data downloaded from https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases

**Note: recovered data for US  available only until Dec 13, 2020**

In [103]:
infected_cases_data_path = "data/countries/time_series_covid19_confirmed_global.csv"
recovered_cases_data_path = "data/countries/time_series_covid19_recovered_global.csv"
deceased_cases_data_path = "data/countries/time_series_covid19_deaths_global.csv"

infected_total = CSV.read(infected_cases_data_path, DataFrame)
infected_US = infected_total[infected_total[!, :"Country/Region"] .== "US", :]
infected = select(infected_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

recovered_total = CSV.read(recovered_cases_data_path, DataFrame)

recovered_US = recovered_total[recovered_total[!, :"Country/Region"] .== "US", :]
recovered = select(recovered_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required


deceased_total = CSV.read(deceased_cases_data_path, DataFrame)
deceased_US = deceased_total[deceased_total[!, :"Country/Region"] .== "US", :]
deceased = select(deceased_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"]))#remove columns that are not required


# #convert to matrices

infected = Matrix(infected)'[1:327]
recovered = Matrix(recovered)'[1:327] #considering only until Dec 13 2020.
deceased = Matrix(deceased)'[1:327]

infected = get_actual_values_from_cumulatives(infected)
recovered = get_actual_values_from_cumulatives(recovered)
deceased = get_actual_values_from_cumulatives(deceased)

327-element Vector{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    ⋮
 2830.0
 2969.0
 2612.0
 2390.0
 1403.0
 1559.0
 2583.0
 3181.0
 2986.0
 3381.0
 2533.0
 1659.0

Algorithm:

Equations:
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

S̃(t + 1) = S̃(t) - β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t))
Ĩ(t + 1) = Ĩ(t) + β * (S̃(t) * Ĩ(t)) ÷ (S̃(t) + Ĩ(t)) - γ * Ĩ(t) - ν * Ĩ(t)
R̃(t + 1) = R̃(t) + γ * Ĩ(t)
D(t + 1) = D(t) + α * ν * Ĩ(t)

where
    I(t) = αĨ(t)

    R(t) = αR̃(t)

    S(t) = αS̃(t)

where I(t), R(t) and S(t) are the actual number of infected, recovered and susceptible population at time t and α≥1;
   
    S(t₀) = ωP

where ωϵ[0,1] and P is the total population.

    S̃(t) = (ω ÷ α) * P - Ĩ(t) - R̃(t) - D̃(t)

    Δ(t) = [Ĩ(t + 1)-Ĩ(t); R̃(t + 1)-R̃(t); D̃(t + 1)-D̃(t)]
    Δ(t) = Φ_ωα(t) = [S̃(t)-β*(S̃(t)*Ĩ(t))÷(S̃(t)+Ĩ(t)) -Ĩ(t) -Ĩ(t)÷α; 0 Ĩ(t) 0; 0 0 Ĩ(t)]
    
    Δ̄ = [ρ^(Θ -t₀)Δ(t₀); ρ^(Θ -t₀-1)Δ(t₀+1); ρ^(Θ -t₀-2)Δ(t₀+2)...Δ(Θ)]
    Φ̄_ωα = [ρ^(Θ -t₀)Φ_ωα(t₀); ρ^(Θ -t₀-1)Φ_ωα(t₀+1); ρ^(Θ -t₀-2)Φ_ωα(t₀+2)...Φ_ωα(Θ)]

where ρϵ(0,1)


Steps:
1. Input: data Ĩ(t), R̃(t), and D(t), maximum value ᾱ of α, weighting parameter ρ, and total population P

outputs -> ω⋆, α⋆, β⋆, γ⋆, ν̄⋆


In [52]:
#we have Ĩ(t) and R̃(t) and D(t) in infected, recovered and deceased dataframes.
P = 332902734 #https://www.census.gov/popclock/
ᾱ = 100       # ᾱ*Infected + ᾱ*Recovered + ᾱ*Deceased < ω*Population
ρ = 0.4

0.4

In [106]:
function get_susceptible(ω, α, P, Ĩ, R̃, D)
    population = zeros(Float64, size(Ĩ)[1])
    fill!(population, ω*P/α)
    return population-infected-recovered-deceased
end

get_susceptible (generic function with 1 method)

In [196]:
function invalid(vector_values)
    for (index, value) in enumerate(vector_values)
        if value < 1
            return true
        end
    end
    return false
end

invalid (generic function with 1 method)

In [121]:
function computeΔ̄(Ĩ, R̃, D, ρ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Δ̄ = zeros(Float64, 3Θ)
    for i in 1:Θ-1
        Δ̄[3(i-1)+1] = ρmatrix[i]*(Ĩ[i+1]-Ĩ[i])
        Δ̄[3(i-1)+2] = ρmatrix[i]*(R̃[i+1]-R̃[i])
        Δ̄[3(i-1)+3] = ρmatrix[i]*(D[i+1]-D[i])
    end
    
    Δ̄[3(Θ-1)+1] = ρmatrix[Θ]*(Ĩ[Θ])
    Δ̄[3(Θ-1)+2] = ρmatrix[Θ]*(R̃[Θ])
    Δ̄[3(Θ-1)+3] = ρmatrix[Θ]*(D[Θ])

    return Δ̄
end

computeΔ̄ (generic function with 1 method)

In [205]:
function computeΦ(ω, α, S̃, Ĩ)
    Θ = size(Ĩ)[1]
    
    ρmatrix = zeros(Float64, Θ)
    for i in 1:Θ
        power = Θ-i
        ρmatrix[i] = ρ^power
    end
    
    Φ = zeros(Float64, 3Θ, 3)
    i=1
    while i<Θ
        Φ[i] = ρmatrix[i]*(S̃[i]*Ĩ[i])/ (S̃[i]+Ĩ[i])
        Φ[3*Θ+i] = -1*ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+i] = -1*ρmatrix[i]*Ĩ[i]/α
        Φ[i+1] = 0
        Φ[3*Θ+i+1] = ρmatrix[i]*Ĩ[i]
        Φ[6*Θ+i+1] = 0
        Φ[i+2] = 0
        Φ[3*Θ+i+2] = 0
        Φ[6*Θ+i+2] = ρmatrix[i]*Ĩ[i]
        i+=3
    end
    return Φ
    
end

# S̃ = get_susceptible(0.5, 2, P, infected, recovered, deceased)
# computeΦ(0.5, 2, S̃, infected)


computeΦ (generic function with 1 method)

In [204]:
function computeParameters(Φ̄, Δ̄)
    pseudo_inv = pinv(Φ̄)
    params = pseudo_inv * Δ̄
    return params
end

computeParameters (generic function with 1 method)

In [199]:
function error_(Δ̄, Φ̄, params)
    param_matrix = zeros(Float64, 3, 1)
    param_matrix[1] = params[1]
    param_matrix[2] = params[2]
    param_matrix[3] = params[3]
    
    diff = Δ̄-(Φ̄*param_matrix)
    return norm(diff)^2
end

# M = [1.5 1.3; 1.2 1.9]
# N = [2,4]
# params = computeParameters(M, N)
# error_(N, M, params)
# # print(params)


error_ (generic function with 1 method)

In [213]:
function sir_model(Ĩ, R̃, D, P, ᾱ, ρ)
    e = Inf
    Δ̄ = computeΔ̄(Ĩ, R̃, D, ρ)
    ω_ = Inf
    α_ = Inf
    β_ = Inf
    γ_ = Inf
    ν̃_ = Inf
    ω = 0 
    
    while ω ≤ 1
        α = 1
        while α ≤ ᾱ
            S̃ = get_susceptible(ω, α, P, Ĩ, R̃, D)
            if invalid(S̃)
                break
            end
            
            Φ̄ = computeΦ(ω, α, S̃, Ĩ)
            params = computeParameters(Φ̄, Δ̄)
            β = params[1]
            γ = params[2]
            ν̃ = params[3]
            if(β<0 || γ<0 || ν̃<0)
                print(ω)
                print(" ")
                println(α)
            end
            error_value = error_(Δ̄, Φ̄, params)
            if error_value < e
                e = error_value
                ω_ = ω
                α_ = α
                β_ = β
                γ_ = γ
                ν̃_ = ν̃
            end
            α+=10
        end
        ω+=0.05
    end
    return ω_, α_, β_, γ_, ν̃_
end

sir_model (generic function with 1 method)

In [214]:
sir_model(infected, recovered, deceased, P, ᾱ, ρ)

0.05 1
0.05 11
0.05 21
0.05 31
0.05 41
0.1 1
0.1 11
0.1 21
0.1 31
0.1 41
0.1 51
0.1 61
0.1 71
0.1 81
0.15000000000000002 1
0.15000000000000002 11
0.15000000000000002 21
0.15000000000000002 31
0.15000000000000002 41
0.15000000000000002 51
0.15000000000000002 61
0.15000000000000002 71
0.15000000000000002 81
0.15000000000000002 91
0.2 1
0.2 11
0.2 21
0.2 31
0.2 41
0.2 51
0.2 61
0.2 71
0.2 81
0.2 91
0.25 1
0.25 11
0.25 21
0.25 31
0.25 41
0.25 51
0.25 61
0.25 71
0.25 81
0.25 91
0.3 1
0.3 11
0.3 21
0.3 31
0.3 41
0.3 51
0.3 61
0.3 71
0.3 81
0.3 91
0.35 1
0.35 11
0.35 21
0.35 31
0.35 41
0.35 51
0.35 61
0.35 71
0.35 81
0.35 91
0.39999999999999997 1
0.39999999999999997 11
0.39999999999999997 21
0.39999999999999997 31
0.39999999999999997 41
0.39999999999999997 51
0.39999999999999997 61
0.39999999999999997 71
0.39999999999999997 81
0.39999999999999997 91
0.44999999999999996 1
0.44999999999999996 11
0.44999999999999996 21
0.44999999999999996 31
0.44999999999999996 41
0.44999999999999996 51
0.449999

(0.05, 1, -7.326769156088313e-88, -4.4365124485511816e-88, -2.1590487456170956e-89)

In [ ]:
function store_data(infected_df,recovered_df,deceased_df)
    infected_cases_data_path = "data/preprocessed/countries/time_series_covid19_confirmed_global.csv"
    recovered_cases_data_path = "data/preprocessed/countries/time_series_covid19_recovered_global.csv"
    deceased_cases_data_path = "data/preprocessed/countries/time_series_covid19_deaths_global.csv"

    CSV.write(infected_cases_data_path, infected_df)
#     infected_US = infected_total[infected_total[!, :"Country/Region"] .== "US", :]
#     infected = select(infected_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(recovered_cases_data_path, recovered_df)
#     recovered_US = recovered_total[recovered_total[!, :"Country/Region"] .== "US", :]
#     recovered = select(recovered_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"])) #remove columns that are not required

    CSV.write(deceased_cases_data_path, deceased_df)
#     deceased_US = deceased_total[deceased_total[!, :"Country/Region"] .== "US", :]
#     deceased = select(deceased_US, Not([:"Province/State", :"Lat", :"Long", :"Country/Region"]))#remove columns that are not required


end

In [35]:
#compute S̃, Ĩ and R̃ of all time t.


# if !isfile("storage/preprocessed_data.csv")
#     for t = 1:length(infected_total)
#         S̃[t] = [ω / α] * P - Ĩ[t] - R̃[t] - D̃[t]
#         Ĩ[t + 1] = Ĩ[t] + β * [[S̃[t] * Ĩ[t]] / [S̃[t] + Ĩ[t]]] - γ * Ĩ[t] - ν * Ĩ[t]
#         R̃[t + 1] = R̃[t] + γ * Ĩ[t]
#         D[t + 1] = D[t] + α * ν * Ĩ[t]
#     store_data(infected_df,recovered_df,deceased_df)
# end

# Store to avoid repeated computing.

LoadError: syntax: "(t + 1)" is not a valid function argument name around In[35]:6